In [ ]:
import numpy as np
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import History, ModelCheckpoint

# Set random seed for reproducibility
np.random.seed(250)

# Hyperparameters
MINI_BATCH_SIZE = 100
NUM_EPOCHS = 10
INPUT_SIZE = 252
NUM_UNITS = 256
NUM_LAYERS = 4
NUM_CLASSES = 88

# File paths
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Datasplit/data/"
WEIGHTS_DIR = "/content/drive/MyDrive/Colab Notebooks/Datasplit/weights/"

def load_data(data_path, batch_count, prefix):
    """Loads data batches from a directory, concatenating them along axis 0."""
    data, labels = [], []
    for i in range(batch_count):
        print(f"Loading batch {i+1}/{batch_count} - {prefix}")
        data.append(np.load(os.path.join(data_path, f"{i}{prefix}_X.npy")))
        labels.append(np.load(os.path.join(data_path, f"{i}{prefix}_y.npy")))
    return np.concatenate(data, axis=0), np.concatenate(labels, axis=0)

print("Building model...")
model = Sequential([
    Dense(NUM_UNITS, input_shape=(INPUT_SIZE,), activation='relu', kernel_initializer='he_normal'),
    Dropout(0.2)
])

for i in range(1, NUM_LAYERS):
    print(f"Adding layer {i+1} with {NUM_UNITS} units")
    model.add(Dense(NUM_UNITS, activation='relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.2))

print("Adding output layer for classification")
model.add(Dense(NUM_CLASSES, activation='sigmoid', kernel_initializer='he_normal'))


model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])


X_val, y_val = load_data(os.path.join(DATA_DIR, "dev"), batch_count=1, prefix="dev")


num_train_batches = len([name for name in os.listdir(os.path.join(DATA_DIR, "train"))]) // 2
X_train, y_train = load_data(os.path.join(DATA_DIR, "train"), num_train_batches, prefix="train")


history = History()
checkpointer = ModelCheckpoint(filepath=os.path.join(WEIGHTS_DIR, "weights.keras"), verbose=1, save_best_only=False)


print("Starting training...")
history = model.fit(
    X_train, y_train,
    batch_size=MINI_BATCH_SIZE,
    epochs=NUM_EPOCHS,
    validation_data=(X_val, y_val),
    verbose=1,
    callbacks=[history, checkpointer]
)


with open("training_log.txt", "w") as log_file:
    log_file.write(str(history.history) + "\n")

print("Training completed and log saved.")


In [ ]:
import numpy as np
import os
from keras.models import load_model

# Set random seed for reproducibility
np.random.seed(250)

# Hyperparameters and paths
MINI_BATCH_SIZE = 100
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Datasplit/data/test/"
WEIGHTS_DIR = "/content/drive/MyDrive/Colab Notebooks/Datasplit/weights/"

# Initialize lists to store data
X, y = [], []

# Load test data
def load_test_data(data_dir, num_batches):
    X_test, y_test = [], []
    for i in range(num_batches):
        print(f"Loading batch {i+1}/{num_batches}")
        X_test.append(np.load(f"{data_dir}{i}test_X.npy"))
        y_test.append(np.load(f"{data_dir}{i}test_y.npy"))
    return np.concatenate(X_test, axis=0), np.concatenate(y_test, axis=0)

num_test_batches = len([name for name in os.listdir(DATA_DIR)]) // 2
print("Loading test data...")
X, y = load_test_data(DATA_DIR, num_test_batches)

# Load model
model = load_model(os.path.join(WEIGHTS_DIR, "weights.keras"))

# Prediction and accuracy functions
def calculate_accuracy(predictions, labels):
    TP = np.count_nonzero(np.logical_and(predictions == 1, labels == 1))
    FN = np.count_nonzero(np.logical_and(predictions == 0, labels == 1))
    FP = np.count_nonzero(np.logical_and(predictions == 1, labels == 0))
    if TP + FN > 0:
        recall = TP / float(TP + FN)
        precision = TP / float(TP + FP)
        accuracy = 100 * TP / float(TP + FP + FN)
        f_measure = 100 * 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    else:
        recall, precision, accuracy, f_measure = 0, 0, 0, 0
    return accuracy, f_measure, recall, precision

# Make predictions
print("Predicting with model...")
predictions = model.predict(X, batch_size=MINI_BATCH_SIZE, verbose=1).round()
predictions[predictions > 1] = 1
np.save(f"{WEIGHTS_DIR}predictions_CQT_DNN", predictions)

# Calculate pre-processed accuracy
print("Calculating pre-processed accuracy...")
pre_accuracy, pre_f_measure, pre_recall, pre_precision = calculate_accuracy(predictions, y)
print(f"F-measure: {pre_f_measure}")
print(f"Accuracy: {pre_accuracy}")

with open(os.path.join(WEIGHTS_DIR, "Accuracy_LSTM.lst"), "w") as main_data:
    main_data.write(f"R = {pre_recall:.6f}\n")
    main_data.write(f"P = {pre_precision:.6f}\n")
    main_data.write(f"A = {pre_accuracy:.6f}\n")
    main_data.write(f"F = {pre_f_measure:.6f}\n")


print("Results saved to Accuracy_LSTM.lst")
